# A bunch of paintings

In this notebook, I'll just make a bunch of paintings using photos I have on hand. I wanna wrap my head around all the different parameters of the model.

In [1]:
# Set working directory
%cd ..

/work


In [2]:
# Import libraries
import torch
from attrdict import AttrDict
from painter import ProgressivePainter
from demo_prog import optimize_x
from IPython.display import Video

# Decide which device we want to run on
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
! rm -rf output/tiwaiwaka

In [4]:
pic_name = 'tiwaiwaka'

In [5]:
# Set parameters
painter_args = AttrDict(
    img_path                = f'./test_images/{pic_name}.jpg',
    renderer                = 'oilpaintbrush',
    canvas_color            = 'white',
    canvas_size             = 512,
    keep_aspect_ratio       = True,
    max_m_strokes           = 4000,
    max_divide              = 20,
    beta_L1                 = 1.0,
    with_ot_loss            = False,
    beta_ot                 = 0.1,
    net_G                   = 'zou-fusion-net',
    renderer_checkpoint_dir = r'./checkpoints_G_oilpaintbrush',
    lr                      = 2e-4,
    output_dir              = f'./output/{pic_name}',
    disable_preview         = True
)

# Initialise a progressive painter
pt = ProgressivePainter(args=painter_args)

initialize network with normal


In [6]:
%%time
optimize_x(pt)

loading renderer from pre-trained checkpoint...
iteration step 499, G_loss: 0.04372, step_acc: 22.43155, grid_scale: 20 / 20, strokes: 1 / 1
saving stroke parameters...
Saving video file to: ./output/tiwaiwaka/tiwaiwaka_animated.mp4
saving input photo...
saving final rendered result...
ffmpeg -i ./output/tiwaiwaka/tiwaiwaka_animated.mp4 -vcodec libx264 ./output/tiwaiwaka/tiwaiwaka.mp4
CPU times: user 1h 43s, sys: 3min 1s, total: 1h 3min 45s
Wall time: 25min 36s


In [7]:
Video(f"output/{pic_name}/{pic_name}.mp4", embed=True)